In [15]:
from summarization_algorithm import *
from preprocessing_algorithms import *

In [16]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
import pandas as pd
data = pd.read_csv('DUC/Duc_first_reference/main_dataset/Duc_dataset_first_ref_summary.csv')
text = data.iloc[1,0]
text

"\r\nHonduras braced for potential catastrophe Tuesday as Hurricane Mitch \r\nroared through the northwest Caribbean, churning up high waves and \r\nintense rain that sent coastal residents scurrying for safer ground. \r\nPresident Carlos Flores Facusse declared a state of maximum alert \r\nand the Honduran military sent planes to pluck residents from their \r\nhomes on islands near the coast. At 0900 GMT Tuesday, Mitch was 95 \r\nmiles (152 kilometers) north of Honduras, near the Swan Islands. With \r\nwinds near 180 mph (289 kph), and even higher gusts, it was a Category \r\n5 monster _ the highest, most dangerous rating for a storm. The 350-mile \r\n(560-kilometer) wide hurricane was moving west at 8 mph (12 kph). \r\n``Mitch is closing in,'' said Monterrey Cardenas, mayor of Utila, \r\nan island 20 miles (32 kilometers) off the Honduran coast. ``And God \r\nhelp us.'' Mitch posed no immediate threat to the United States, forecasters \r\nsaid, but was expected to remain in the north

In [3]:
# # define the input text
# #
# # load the GPT-2 tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# # tokenize the input text into sentences
# sentences = text.split('. ')

# # compute the embedding of each sentence using GPT-2
# embeddings = []
# for sentence in sentences:
#     input_ids = tokenizer.encode(sentence, return_tensors='pt')
#     with torch.no_grad():
#         output = model(input_ids)[0][:, -1, :]
#         embeddings.append(output)
# # compute the similarity between each sentence embedding and the embedding of the entire text
# text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
# similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0), text_embedding)

# # rank the sentences based on their similarity score
# sentence_scores = list(zip(sentences, similarity_scores))
# sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

# # select the top N sentences to form the summary
# N = 5
# summary = [s[0] for s in sentence_scores[:N]]

In [2]:
def gpt2_extractive_summarization(filtered_sentences,text, num_sentences):
    # load the GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')

    # tokenize the input text into sentences
    sentences = text.split('. ')
    print(sentences)

    # compute the embedding of each sentence using GPT-2
    embeddings = []
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            output = model(input_ids)[0][:, -1, :]
            embeddings.append(output)

            

    # compute the similarity between each sentence embedding and the embedding of the entire text
    text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
    return embeddings,text_embedding
    similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())


    # rank the sentences based on their similarity score
    sentence_scores = list(zip(text, similarity_scores))
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # select the top N sentences to form the summary
    summary = [s[0] for s in sentence_scores[:num_sentences]]
    return '. '.join(summary)


In [3]:
filtered_sentences

NameError: name 'filtered_sentences' is not defined

In [4]:
x,y = gpt2_extractive_summarization('x',text, num_sentences=5)


NameError: name 'text' is not defined

In [41]:
import heapq


In [42]:
def gpt2_extractive_summarization(sentences, filtered_sentences, size):
    # load the GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')

    # compute the embedding of each sentence using GPT-2
    embeddings = []
    for sentence in filtered_sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            output = model(input_ids)[0][:, -1, :]
            embeddings.append(output)

    # compute the similarity between each sentence embedding and the embedding of the entire text
    text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
    similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())

    # rank the sentences based on their similarity score
    sentence_scores = list(zip(sentences, similarity_scores))
    top_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)[:size]

    # select the top N sentences to form the summary
    summary = [s[0] for s in sentence_scores if s in top_scores]
    scores = [s[1] for s in sentence_scores if s in top_scores]
    return scores, summary


In [43]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Original sentences
original_sentences = ["The quick brown fox jumps over the lazy dog.",
                       "This is a sentence.",
                       "Another sentence here.",
                       "This is a short sentence.",
                       "The dog is lazy."]

# Preprocess the sentences
filtered_sentences = []
for sentence in original_sentences:
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence.lower())
    
    # Remove stopwords and lemmatize the remaining words
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    
    # Convert the filtered words back to a sentence and add it to the list of filtered sentences
    filtered_sentence = ' '.join(filtered_words)
    filtered_sentences.append(filtered_sentence)

In [20]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zeyad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
original_sentences

['The quick brown fox jumps over the lazy dog.',
 'This is a sentence.',
 'Another sentence here.',
 'This is a short sentence.',
 'The dog is lazy.']

In [22]:
gpt2_extractive_summarization(filtered_sentences,original_sentences,5)

c:\Users\Zeyad\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zeyad\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


([array([0.9998806], dtype=float32),
  array([0.99981004], dtype=float32),
  array([0.99971485], dtype=float32),
  array([0.9996187], dtype=float32),
  array([0.9989705], dtype=float32)],
 ['short sentence .',
  'sentence .',
  'another sentence .',
  'dog lazy .',
  'quick brown fox jump lazy dog .'])

In [23]:
from preprocessing_algorithms import *

In [29]:
article_str_sentences, processed_sentences = preprocessing_text_with_spacy(text, False, False)

In [31]:
article_str_sentences

['\r\nHonduras braced for potential catastrophe Tuesday as Hurricane Mitch \r\nroared through the northwest Caribbean, churning up high waves and \r\nintense rain that sent coastal residents scurrying for safer ground. \r\n',
 'President Carlos Flores Facusse declared a state of maximum alert \r\nand the Honduran military sent planes to pluck residents from their \r\nhomes on islands near the coast.',
 'At 0900 GMT Tuesday, Mitch was 95 \r\nmiles (152 kilometers) north of Honduras, near the Swan Islands.',
 'With \r\nwinds near 180 mph (289 kph), and even higher gusts, it was a Category \r\n5 monster _',
 'the highest, most dangerous rating for a storm.',
 'The 350-mile \r\n(560-kilometer) wide hurricane was moving west at 8 mph (12 kph). \r\n',
 "``Mitch is closing in,'' said Monterrey Cardenas, mayor of Utila, \r\nan island 20 miles (32 kilometers) off the Honduran coast.",
 "``And God \r\nhelp us.''",
 'Mitch posed no immediate threat to the United States, forecasters \r\nsaid, but 

In [32]:
processed_sentences

['honduras braced for potential catastrophe tuesday as hurricane mitch roared through the northwest caribbean churning up high waves and intense rain that sent coastal residents scurrying for safer ground',
 'president carlos flores facusse declared a state of maximum alert and the honduran military sent planes to pluck residents from their homes on islands near the coast',
 'at 0900 gmt tuesday mitch was 95 miles 152 kilometers north of honduras near the swan islands',
 'with winds near 180 mph 289 kph and even higher gusts it was a category 5 monster',
 'the highest most dangerous rating for a storm',
 'the 350-mile 560-kilometer wide hurricane was moving west at 8 mph 12 kph',
 'mitch is closing in said monterrey cardenas mayor of utila an island 20 miles 32 kilometers off the honduran coast',
 'and god help us',
 'mitch posed no immediate threat to the united states forecasters said but was expected to remain in the northwest caribbean for five days',
 'the u s national weather ser

In [44]:
gpt2_extractive_summarization(processed_sentences,article_str_sentences,5)

([array([0.9984894], dtype=float32),
  array([0.9986974], dtype=float32),
  array([0.998545], dtype=float32),
  array([0.9984776], dtype=float32),
  array([0.9986681], dtype=float32)],
 ['la ceiba officials appealed for pure water for those in shelters and some residents set out plastic buckets to collect rainwater',
  'more than 20 000 tourists had abandoned cancun and nearby resort areas leaving hotels at about 20 percent of capacity',
  'guatemala reported 100 storm-related deaths',
  'six people died in southern mexico and seven in costa rica',
  'the estimated number of homeless dropped from 580 000 to 569 000 thursday'])